In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
from collections import Counter
import itertools
import os
import sys
from joblib import Parallel, delayed
import multiprocessing
import matplotlib

In [ ]:
# Upload Gene expression database, pathway database, TF list
all_pathways_lst= pd.read_table("./aracyc_pathways_input1.txt")
all_gene_exp = pd.read_csv("../../data/arabidopsis_drout_leafs/Dataset_drought_stress_seq17_2016st_136samples.csv")
all_tf_lst = pd.read_csv("../../data/arabidopsis/AT_tf_list.csv")


#Generate pathway and tf expression files
PW_exp_data = pd.merge(all_pathways_lst,all_gene_exp,left_on='PW_gene',right_on='Gene')
TF_exp_data = pd.merge(all_tf_lst,all_gene_exp,left_on="Gene",right_on='Gene')

PW_exp_data = PW_exp_data.drop(['PW_gene','PW_id','PW_name','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8'], 1)

PW_exp_data.drop_duplicates(subset='Gene',inplace=True)

TF_exp_data.drop_duplicates(subset='Gene',inplace=True)

PW_exp_data.to_csv("PW_exp_data.csv")
TF_exp_data.to_csv("TF_exp_data.csv")

In [ ]:
#Better to run this in a server
def processInput(chunk):
    """ This block computes MI between all pairs of PW genes and TFs. Run this part in a server and obtain the results.csv
    file for further processing. pathway file was split to 100 gene chunks to avoid momory overflow errors. The code to 
    do chunking is shown below."""
    
    tempChunkPWs = PW_exp_data.iloc[chunk]
    combined_temp = pd.concat([tempChunkPWs, TF_exp_data])
    combined_temp.to_csv("chunk"+str(chunk[-1]+1)+".csv",header=False)
    command = "java -jar MINE.jar chunk"+str(chunk[-1]+1)+".csv -pairsBetween "+str(chunk[-1]+1)+" notify=500 >/dev/null"
    os.system(command)
    os.system("rm chunk"+str(chunk[-1]+1)+".csv")
    os.system("rm chunk"+str(chunk[-1]+1)+".csv,between[break=100],cv=0.0,B=n^0.6,Status.txt")
    temp_results_df = pd.read_csv("./chunk"+str(chunk[-1]+1)+".csv,between[break=1000],cv=0.0,B=n^0.6,Results.csv",header=False)
    #os.system("rm ./chunk"+str(chunk[-1]+1)+".csv,between[break=1000],cv=0.0,B=n^0.6,Results.csv")
    return temp_results_df


#pw_exp_data = pd.read_csv("pw_exp_selected.csv")
#TF_exp_data = pd.read_table("./At_Stem_TF1622TF_uniq_exp_HW.txt")
data = range(PW_exp_data.shape[0])
chunks = [data[x:x+100] for x in xrange(0, len(data), 100)]
num_cores = 8 #multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(processInput)(chunk) for chunk in chunks)

results.to_csv("results.csv",header=False)

In [ ]:
#Get a filterd dictionary of pathways with genes > 20. I analysed pathways which have atleast 20 genes.
PW_exp = pd.read_csv("./pathway_exp.csv")
temp = PW_exp[['PW_name','PW_gene']]
temp_dic = {pw: pwg["PW_gene"].tolist() for pw,pwg in temp.groupby("PW_name")}
filtered_dict = {k:v for k,v in temp_dic.iteritems() if len(list(set(v))) > 20}

In [ ]:
# This cell to get the association dataframe in recudes PW set and all TFs
selected_PWg = []
for key in filtered_dict:
    selected_PWg.append(filtered_dict[key])
chain = itertools.chain(*selected_PWg)
selected_PWg = set(list(chain))

raw_data = {'pw_gene': list(selected_PWg)}

pw_gene_df = pd.DataFrame(raw_data,columns=['pw_gene'])



pw_exp_data = pd.merge(pw_gene_df,gene_exp,left_on='pw_gene',right_on='Gene')

#pw_exp_data.to_csv("pw_exp_selected.csv")
pw_exp_data = pd.read_csv("pw_exp_selected.csv")

#pw_exp_data.drop('pw_gene', axis=1, inplace=True)

data = range(pw_exp_data.shape[0])

def processInput(chunk):
    tempChunkPWs = pw_exp_data.iloc[chunk]
    combined_temp = pd.concat([tempChunkPWs, TF_exp])
    combined_temp.to_csv("chunk"+str(chunk[-1]+1)+".csv",header=False,index=False)
    #command = "java -jar MINE.jar chunk"+str(chunk[-1]+1)+".csv -pairsBetween "+str(len(chunk))+" notify=500 >/dev/null"
    #os.system(command)
    #os.system("rm chunk"+str(chunk[-1]+1)+".csv")
    #os.system("rm chunk"+str(chunk[-1]+1)+".csv,between[break=100],cv=0.0,B=n^0.6,Status.txt")
    #temp_results_df = pd.read_csv("./chunk"+str(chunk[-1]+1)+".csv,between[break=1000],cv=0.0,B=n^0.6,Results.csv",header=False)
    #os.system("rm ./chunk"+str(chunk[-1]+1)+".csv,between[break=1000],cv=0.0,B=n^0.6,Results.csv")

chunks = [data[x:x+100] for x in xrange(0, len(data), 100)]
#num_cores = 8 #multiprocessing.cpu_count()
#results = Parallel(n_jobs=num_cores)(delayed(processInput)(chunk) for chunk in chunks)


for chunk in chunks[0:17]:
    print(chunk[-1]+1)
    print ("./chunk"+str(chunk[-1]+1)+".csv,between[break=100],cv=0.0,B=n^0.6,Results.csv")
    temp_results_df = pd.read_csv("./chunk"+str(chunk[-1]+1)+".csv,between[break=100],cv=0.0,B=n^0.6,Results.csv")
    pd.concat([association,temp_results_df])

association.to_csv("all_PW_TF_association.csv")

In [ ]:
association = association[association['MIC (strength)'] > 0.5]

In [ ]:
selected_top_TFs_forPW = pd.DataFrame(columns=[u'pw_index', u'pw', u'pwg', u'X var', u'Y var', u'MIC (strength)',
       u'MIC-p^2 (nonlinearity)', u'MAS (non-monotonicity)',
       u'MEV (functionality)', u'MCN (complexity)', u'Linear regression (p)'])
ind = 0
for pw in filtered_dict:
    #if(ind==1):
     #   break
    ind = ind + 1
    print(ind,end=",")
    print(pw,end=",")
    print(len(filtered_dict[pw]))
    for pwg in filtered_dict[pw]:
        temp1 = association[association['X var']==pwg]
        temp1 = temp1.sort_values(by='MIC (strength)',ascending=False)
        temp1.reset_index(drop=True,inplace=True)
        nRows = temp1.shape[0]
        nRows
        temp0 = pd.DataFrame([[ind]*nRows,[pw]*nRows,[pwg]*nRows]).transpose()
        temp0.reset_index(drop=True, inplace=True)
        temp0.columns=['pw_index','pw','pwg']
        selected_df = pd.concat( [temp0, temp1], axis=1)
        #selected_df2 = pd.concat([row, selected_df], axis=1,ignore_index="True") THis line does not work for concat dfs !!!!
        selected_top_TFs_forPW = pd.concat([selected_top_TFs_forPW,selected_df])
    

In [ ]:

selected_top_TFs_forPW.to_csv("/Users/chathura/Desktop/MIC_cluster_Analysis/all_PW_1622TF_MIC.csv")

In [ ]:
ID2sym = pd.read_table("./AT_ID_SYM.txt",header=None)
ID2sym.columns = ["pwg","pwg_sym"]
temp = pd.merge(selected_top_TFs_forPW,ID2sym,on='pwg')

In [ ]:
chunks = [data[x:x+100] for x in xrange(0, len(data), 100)]

In [ ]:
association = pd.read_csv("./chunk1790.csv,between[break=90],cv=0.0,B=n^0.6,Results.csv")
for chunk in chunks[0:17]:
    print(chunk[-1]+1)
    print ("./chunk"+str(chunk[-1]+1)+".csv,between[break=100],cv=0.0,B=n^0.6,Results.csv")
    temp_results_df = pd.read_csv("./chunk"+str(chunk[-1]+1)+".csv,between[break=100],cv=0.0,B=n^0.6,Results.csv")
    association = pd.concat([association,temp_results_df])

In [ ]:
#Recreate the original PW to PWG-TF associations
association = pd.read_csv("./allPW_TF_MIC_association.csv")
selected_top_TFs_forPW = pd.DataFrame(columns=[u'pw_index', u'pw', u'pwg', u'X var', u'Y var', u'MIC (strength)',
       u'MIC-p^2 (nonlinearity)', u'MAS (non-monotonicity)',
       u'MEV (functionality)', u'MCN (complexity)', u'Linear regression (p)'])
pw_index = 0
top_TFs = 100
for pw in filtered_dict:
    pw_index = pw_index + 1
    for pwg in set(filtered_dict[pw]):
        
        row = pd.DataFrame([[pw_index]*top_TFs,[pw]*top_TFs,[pwg]*top_TFs]).transpose()
        cols = ['pw_index', 'pw','pwg']
        row.columns = cols
        
        temp = association[association['X var'] == pwg]
        temp = temp.sort_values(by='MIC (strength)',ascending=False)
        selected_df = temp.iloc[0:no_top_TFs]
        
        row.reset_index(drop=True, inplace=True)
        selected_df.reset_index(drop=True, inplace=True)
        selected_df2 = pd.concat( [row, selected_df], axis=1)
        selected_df2 = selected_df2.dropna()
        #selected_df2 = pd.concat([row, selected_df], axis=1,ignore_index="True") THis line does not work for concat dfs !!!!
        selected_top_TFs_forPW = pd.concat([selected_top_TFs_forPW,selected_df2])
        
            
        
        
        
        
        

In [ ]:
selected_top_TFs_forPW.shape

In [ ]:
ID2sym = pd.read_table("./AT_ID_SYM.txt",header=None)
ID2sym.columns = ["pwg","pwg_sym"]

temp = pd.merge(selected_top_TFs_forPW,ID2sym,on='pwg')
ID2sym.columns = ['Y var','tf_sym']
final = pd.merge(temp,ID2sym,on='Y var')

In [ ]:
final.to_csv("/Users/chathura/Desktop/MIC_cluster_Analysis/all_PW_1622TF_MIC_annotated.csv")

In [ ]:
#This is to get the data ready for heatmap and clustering

pw_df = final[final['pw_index']==1]
tf_list = []
for pwg in set(pw_df['pwg_sym']):
    pwg_df = pw_df[pw_df['pwg_sym']==pwg]
    pwg_df = pwg_df.sort_values(by='MIC (strength)',ascending=False).iloc[0:10] # find a better way to do the cutoff
    tf_list.append(list(pwg_df['tf_sym']))
    
tf_list = list(set(list(itertools.chain.from_iterable(tf_list))))
pw_list = list(set(pw_df['pwg_sym']))
df = pd.DataFrame(index=range(0,len(tf_list)), columns=pw_list)
#df = df.fillna(0) # with 0s rather than NaNs 
for pw in pw_list:
    pwg_df = pw_df[pw_df['pwg_sym']==pw]
    pwg_df = pwg_df.sort_values(by='MIC (strength)',ascending=False).iloc[0:10]
    for tf_i in range(len(tf_list)):
        tf_sym = tf_list[tf_i]
        if(tf_sym in list(pwg_df['tf_sym'])):
            idx = list(pwg_df['tf_sym']).index(tf_sym)
            df.at[tf_i,pw] = pwg_df.iloc[idx]['MIC (strength)']
        else:
            pass
        
df = df.fillna(0)

            
    

        